In [2]:
import torch
import numpy as np

In [14]:
def maxPool2D(input, kernel_size=[2,2], stride=1):
#     input: input matrix
    in_batch = input.size(0)
    in_channel = input.size(1)
    in_height = input.size(2)
    in_width = input.size(3)
    
    k_height = kernel_size[0]
    k_width = kernel_size[1]

    out_height = int((in_height - k_height) / stride) + 1
    out_width = int((in_width - k_width) / stride) + 1
    out = torch.zeros((in_batch, in_channel, out_height, out_width))

    for a in range(in_batch):
        for k in range(in_channel):
            for i in range(out_height):
                for j in range(out_width):
                    start_i = i * stride
                    start_j = j * stride
                    end_i = start_i + k_height
                    end_j = start_j + k_width
                    out[a, k, i, j] = torch.max(input[a, k, start_i: end_i, start_j: end_j])
    return out
                    
                    
input = torch.randn([1, 3, 32, 32])
print(maxPool2D(input).shape)
model = torch.nn.Sequential(
    torch.nn.MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False)
)
print(maxPool2D(input) - model(input))

torch.Size([1, 3, 31, 31])
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])


In [13]:
def averagePool2D(input, kernel_size=[2,2], stride=1):
    #     input: input matrix
    in_batch = input.size(0)
    in_channel = input.size(1)
    in_height = input.size(2)
    in_width = input.size(3)
    
    k_height = kernel_size[0]
    k_width = kernel_size[1]

    out_height = int((in_height - k_height) / stride) + 1
    out_width = int((in_width - k_width) / stride) + 1
    out = torch.zeros((in_batch, in_channel, out_height, out_width))

    for a in range(in_batch):
        for k in range(in_channel):
            for i in range(out_height):
                for j in range(out_width):
                    start_i = i * stride
                    start_j = j * stride
                    end_i = start_i + k_height
                    end_j = start_j + k_width
                    out[a, k, i, j] = torch.mean(input[a, k, start_i: end_i, start_j: end_j])
    return out
                    
                    
input = torch.randn([1, 3, 32, 32])
print(averagePool2D(input).shape)
model = torch.nn.Sequential(
    torch.nn.AvgPool2d(kernel_size=2, stride=1, padding=0, ceil_mode=False, count_include_pad=True, divisor_override=None)
)
print(averagePool2D(input) - model(input))

torch.Size([1, 3, 31, 31])
tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 2.9802e-08,  0.0000e+00,  0.0000e+00,  ...,  1.1921e-07,
           -5.9605e-08,  0.0000e+00],
          [ 0.0000e+00, -2.9802e-08, -2.9802e-08,  ...,  0.0000e+00,
            0.0000e+00, -2.9802e-08],
          ...,
          [ 0.0000e+00,  0.0000e+00,  5.9605e-08,  ..., -5.9605e-08,
           -5.9605e-08,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -1.4901e-08,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-2.9802e-08,  0.0000e+00,  2.9802e-08,  ...,  0.0000e+00,
           -2.9802e-08,  0.0000e+00]],

         [[ 5.9605e-08,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -2.9802e-08,  ...,  0.0000e+00,
            0.0000e+00,  0.0000

In [15]:
def dilation_kernel(kernel, dilation):
#     kernel : tensor size(N, C, H, W)
#     dilation: int 
    res = torch.zeros([len(kernel), len(kernel[0]), len(kernel[0][0])+(dilation-1)*(len(kernel[0][0])-1), len(kernel[0][0])+(dilation-1)*(len(kernel[0][0])-1)])
    for i in range(len(kernel)):
        for j in range(len(kernel[0])):
            res[i][j] = dilation_kernel_(kernel[i][j], dilation)
    return res

def dilation_kernel_(kernel, dilation):
    dilation_kernel = torch.zeros([len(kernel)+(dilation-1)*(len(kernel)-1), len(kernel)+(dilation-1)*(len(kernel)-1)])
    count = 0
    for i in range(0, len(dilation_kernel), dilation):
        for j in range(0, len(dilation_kernel), dilation):
            dilation_kernel[i][j] = kernel.flatten()[count]
            count+=1
    return dilation_kernel


c = torch.ones([1, 1, 3, 3])
print(c)
print(dilation_kernel(c, 2))

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])
tensor([[[[1., 0., 1., 0., 1.],
          [0., 0., 0., 0., 0.],
          [1., 0., 1., 0., 1.],
          [0., 0., 0., 0., 0.],
          [1., 0., 1., 0., 1.]]]])


In [16]:
def convolution2D(input, kernel, bias, stride=1, dilation=1):
#     input: input matrix(N, C, H, W)
#     kernel: tensor size(N, C, H, W)
#     bias: tensor size(N,)
    if dilation != 1:
        kernel = dilation_kernel(kernel, dilation=dilation)
    out = conv2d_multi_in_out(input, kernel, bias, stride)
    return out


#  one channel and one kernel
# X input K kernel
def conv2d(X, K, stride=1):
    h,w = K.shape
    Y = torch.zeros(int((X.shape[0] - h) / stride) + 1, int((X.shape[1] - w) / stride) + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i * stride : i * stride + h, j * stride : j * stride + w] * K).sum()
    return Y 

#  multi channel and one kernel
def conv2d_mutil_in(X, K, b, stride=1):
    h,w = K.shape[1],K.shape[2]
    value = torch.zeros(int((X[0].shape[0] - h)/ stride) + 1, int((X[0].shape[1]- w) / stride)  + 1)
    for x,k in zip(X,K):
        value = value + conv2d(x, k, stride)
    return value + b

#  multi channel and multi kernel
def conv2d_multi_in_out(input, kernel, bias, stride=1):
    return torch.stack([torch.stack([conv2d_mutil_in(one_input, k, b, stride) for k, b in zip(kernel, bias)]) for one_input in input])
#     return torch.stack([conv2d_mutil_in(input, k, b, stride) for k, b in zip(kernel, bias)])




test = torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
model = torch.nn.Sequential(test)
kernel = test.weight.data
bias = test.bias.data
input = torch.randn([1, 3, 32, 32])

print(model(input).shape)
print(convolution2D(input, kernel, bias, 1, 1).shape)
print(model(input) - convolution2D(input, kernel, bias, 1, 1))

torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 30, 30])
tensor([[[[ 4.4703e-08,  5.9605e-08,  1.4901e-08,  ...,  2.9802e-08,
            5.9605e-08,  1.1921e-07],
          [-1.1921e-07,  0.0000e+00,  0.0000e+00,  ...,  5.9605e-08,
            4.4703e-08,  0.0000e+00],
          [-3.5763e-07,  0.0000e+00,  5.9605e-08,  ..., -2.9802e-08,
            1.7881e-07,  2.2352e-08],
          ...,
          [-5.9605e-08,  0.0000e+00,  1.4901e-08,  ...,  2.9802e-08,
            0.0000e+00, -2.9802e-08],
          [ 1.7881e-07,  1.1921e-07, -1.1921e-07,  ...,  0.0000e+00,
            0.0000e+00, -2.3842e-07],
          [ 0.0000e+00,  1.7881e-07,  0.0000e+00,  ..., -2.9802e-08,
            0.0000e+00,  0.0000e+00]],

         [[ 8.9407e-08,  2.9802e-08,  8.9407e-08,  ..., -8.9407e-08,
           -1.4901e-08,  0.0000e+00],
          [-1.1921e-07,  0.0000e+00, -1.1921e-07,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 1.1921e-07, -2.9802e-08, -2.9802e-08,  ..., -2.9802e-08,
    

In [ ]:
class My_ConvTranspose2d(nn.Module):
    def __init__(self, in_channels=3, out_channels=4, kernel_size=(3,3), stride=2, bias=bias):
        super(My_ConvTranspose2d, self).__init__()
        self.stride = stride
        self.bias = bias
        self.kernel_size = kernel_size
        self.k_height = kernel_size[0]
        self.k_width = kernel_size[1]
        self.in_channels = in_channels
        self.out_channels = out_channels

    def forward(self, x):
        in_batch = x.size(0)
        in_channel = x.size(1)
        in_height = x.size(2)
        in_width = x.size(3)

        out_height = in_height + self.k_height - 1
        out_width = in_width + self.k_width -1
        out = torch.zeros((in_batch, self.out_channels, out_height, out_width))

        for a in range(in_batch):
            for ker in range(self.out_channels):
                for k in range(in_channel):
                    for i in range(in_height):
                        for j in range(in_width):
                            out[a, ker, i:i+self.k_height, j:j+self.k_width] += x[a, k, i, j] * kernel[k][ker]
        for i in range(self.out_channels):
            out[0][i] = out[0][i]+bias[i]
        return out

In [554]:
def flatten(input):
    return input.reshape(-1)

In [598]:
def sigmoid(x):
    return 1 / (1+torch.exp(-x))

In [3]:
def cross_entropy(input, target):
#     input: prediction
#     target: true label    
    log_softmax = logsoftmax(input)
    loss = nn_loss(log_softmax, target)
    return loss.mean()

def logsoftmax(x):
    res = torch.zeros(x.shape)
    for i in range(len(x)):
        exp = torch.exp(x[i])
        sum = torch.sum(exp, dim=0)
        softmax = exp/sum
        res[i] = -torch.log(softmax)
    return res
    
def nn_loss(input, target):
    loss_list = torch.zeros([len(input)])
    for k in range(len(input)):
        loss = 0
        for i in range(target[k].shape[1]):
            for j in range(target.shape[2]):
                index = target[k, i, j]
                num = input[k, index, i ,j]
                loss -= num
        loss_list[k] = -loss / target.shape[1] / target.shape[2]
    return loss_list

input = torch.randn([1, 3, 4, 4])
target = torch.randint(3, (1, 4, 4), dtype=torch.int64)

print(cross_entropy(input, target))
print(torch.nn.functional.cross_entropy(input, target, weight=None,size_average=None, ignore_index=-100, reduce=None, reduction='mean'))

tensor(1.2988)
tensor(1.2988)


In [ ]:
def roi_pool(input=input, boxes=boxes, output_size=(7,7)):
    
    roi_start_h = int(boxes[0][1])
    roi_start_w = int(boxes[0][2])
    roi_end_h = int(boxes[0][3]) 
    roi_end_w = int(boxes[0][4])
    
    roi_width = (roi_end_w - roi_start_w + 1)
    roi_height = (roi_end_h - roi_start_h + 1)

    bin_size_w = roi_width / 7
    bin_size_h = roi_height / 7
    
    self_result = []
    for k in range(3):
        temp = []
        for j in range(7):
            sub_res = []
            for i in range(7):
                sub_res.append(input[..., int(0+bin_size_h*(j)):math.ceil(0+bin_size_h*(j+1)),
                                     int(0+bin_size_w*(i)):math.ceil(0+bin_size_w*(i+1))][0][k].max())
            temp.append(sub_res)
        self_result.append(temp)
    res=torch.Tensor(self_result)
    return res

In [621]:
def mse_loss(input, target):
    square = (input - target)**2
    return square.mean()

input = torch.randn([1, 3, 32, 32])
target = torch.randn([1, 3, 32, 32])


print(torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='mean'))
print(mse_loss(input, target))

tensor(1.9663)
tensor(1.9663)
